<a href="https://colab.research.google.com/github/gabrielcordeiro2/LOL-Champion-Analysis/blob/main/Scrapping_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Important:** if you have any errors:
- go to Runtime > Restart Runtime, or just press Ctrl + M + (.)
- try run all boxes again

**Important:** you need to download and upload the **files folder** located in this repository:
- https://github.com/gabrielcordeiro2/LOL-Champion-Analysis
- or you can directly download using command line:

In [ ]:
!apt install unzip -q
!wget https://tinyurl.com/2p8fpkf7 -q
!unzip 2p8fpkf7

Import necessary files and libs:

In [3]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup
from files.lol_regions import regions
from files.lol_genders import genders
from files.lol_names import correction_roles

### __Champion gender:__

Create Dataframe:

In [2]:
lol_df = pd.read_csv("files/data.csv")

Add gender to respective champions:

In [3]:
for k,v in genders.items():
    lol_df.loc[lol_df["champion"] == k, "gender"] = v

In [ ]:
lol_df

### __Champion roles:__

Get list from API of all champions:

In [5]:
patch = "12.12.1"
ddragon = requests.get(f"http://ddragon.leagueoflegends.com/cdn/{patch}/data/en_US/champion.json").json()
champions_all_info = list(ddragon['data'].items())

Create list from API keys:

In [6]:
champions_keys = []
for info_tuple in champions_all_info:
    champions_keys.append(info_tuple[0])

In [ ]:
champions_keys

Storage information about champíons:

In [8]:
champions_tuple = tuple(champions_all_info)

Create list with champions:

In [9]:
list_champions = []
for i in champions_tuple:
    list_champions.append(i[1]["name"])

Create list with chompions and tags:

In [10]:
list_tags = []
for k,c in zip(champions_keys, list_champions):
    tag_champion = ddragon['data'][k]['tags']
    try:
        list_tags.append([c, f"{tag_champion[0]}, {tag_champion[1]}"])
    except:
        list_tags.append([c, f"{tag_champion[0]}"])

In [ ]:
list_tags

Create Dataframe:

In [12]:
tags_df = pd.DataFrame(list_tags, columns=["champion", "role"])

Apply name correction in dataframe::

In [13]:
for key, value in correction_roles.items():
    lol_df.loc[lol_df["champion"] == key, "champion"] = value

Merge Dataframes:

In [14]:
lol_df = lol_df.merge(tags_df)

In [ ]:
lol_df

### __Champion Stats with Selenium:__

**Note:**
- you need to install Selenium and Kora to proceed.
- Selenium will emulate an hidden browser in your Colab.

In [24]:
!pip3 install --upgrade requests -q
!pip install kora -q
!pip install selenium -q

In [16]:
from kora.selenium import wd
from selenium.webdriver.common.by import By

In [17]:
url = "https://na.op.gg/statistics/champions?hl=en_US&region=global"

wd.get(url)
wd.implicitly_wait(7)
wd.find_element(By.XPATH, '//*[@id="content-container"]/div[2]/table/tbody/tr[118]/td[2]')

html = wd.page_source
wd.close()

Transform html table into dataframe:

In [18]:
stats_df = pd.read_html(html)
stats_df = stats_df[1]

In [ ]:
stats_df

Clean and organize column names:

In [20]:
stats_df.drop(["#", "CS", "Gold", "Games played", "KDA"], inplace=True, axis=1, errors='ignore')
stats_df.rename(
inplace=True,
columns= {"Champion": "champion", 
          "Win rate": "win_rate",
          "Pick ratio per game": "pick_rate",
          "Ban ratio per game": "ban_rate"})
          
stats_df.sort_values("champion", inplace=True, axis=0, ignore_index=True)

Merge Dataframes:

In [21]:
lol_df = lol_df.merge(stats_df, sort=True)

In [ ]:
stats_df

In [ ]:
lol_df

### __Champion Region with Requests:__

In [24]:
def get_json_and_scrap(reg):
    url = f'https://universe-meeps.leagueoflegends.com/v1/en_gb/factions/{reg}/index.json'
    response = requests.get(url).json()

    region_name = response['faction']['name']
    region_members = response['associated-champions']

    for i in region_members:
        champ_name = i['title']
        champs_with_region.append([champ_name, region_name])
    return

Run multi-page scrapping:

In [25]:
champs_with_region = []
for region in regions:  #   ~8 seconds
    get_json_and_scrap(region)

In [ ]:
champs_with_region

Create Dataframe:

In [27]:
scrap_region_df = pd.DataFrame(champs_with_region, columns=["champion","region"])

In [ ]:
scrap_region_df

Create an function to organize dataframes:

In [29]:
def drop_and_sort_rows(dframe):
    dframe.drop_duplicates(subset="champion", keep="first", inplace=True)
    dframe.sort_values(by="champion",ignore_index=True, inplace=True)
    return

Organize and apply name correction:

In [30]:
scrap_region_df.champion.replace("’","'", regex=True, inplace=True)
drop_and_sort_rows(scrap_region_df)

Create template with organized champions:

In [31]:
main_champions = pd.DataFrame({"champion":lol_df.champion.unique()})
drop_and_sort_rows(main_champions)

Merge data to region Dataframe:

In [32]:
full_region_df = main_champions.merge(scrap_region_df, how="left", sort=True)
drop_and_sort_rows(full_region_df)

Add Runeterra for champions without region:

In [33]:
full_region_df.region.loc[full_region_df.region.isnull()] = "Runeterra"

Merge dataframes:

In [34]:
lol_df = lol_df.merge(full_region_df, sort=True)

In [ ]:
full_region_df

In [ ]:
lol_df